In [4]:
import os
from dotenv import load_dotenv

FLASK_PORT = 8080
CLOUD_URL = 'https://pulse-fi.herokuapp.com'

load_dotenv()

def getSecret(secret):
    return os.environ.get(secret)

def isRunningInCloud():
    return os.environ.get('RUNNING') == 'cloud'

def baseUrl():
    if isRunningInCloud():
        return CLOUD_URL
    else:
        return 'http://127.0.0.1:'+str(FLASK_PORT)
    
import requests

# url = baseUrl() + '/heartRate'
# response = requests.get(url)

In [134]:
now_playing_data['item']['artists'][0]


{'external_urls': {'spotify': 'https://open.spotify.com/artist/05YVYeV4HxYp5rrWalvuE1'},
 'href': 'https://api.spotify.com/v1/artists/05YVYeV4HxYp5rrWalvuE1',
 'id': '05YVYeV4HxYp5rrWalvuE1',
 'name': 'Grover Washington, Jr.',
 'type': 'artist',
 'uri': 'spotify:artist:05YVYeV4HxYp5rrWalvuE1'}

In [139]:


#  'disc_number': 1,
#   'duration_ms': 438493,
#   'explicit': False,
#   'external_ids': {'isrc': 'USEE19901162'},
#   'external_urls': {'spotify': 'https://open.spotify.com/track/1ko2lVN0vKGUl9zrU0qSlT'},
#   'href': 'https://api.spotify.com/v1/tracks/1ko2lVN0vKGUl9zrU0qSlT',
#   'id': '1ko2lVN0vKGUl9zrU0qSlT',
#   'is_local': False,
#   'name': 'Just the Two of Us (feat. Bill Withers)',
#   'popularity': 77,
#   'preview_url': 'https://p.scdn.co/mp3-preview/0b8c95f421393a3fca43687498e15ad0b1e1f4d6?cid=19cbb48ee64f4a5d925d12fa24b3ce61',
#   'track_number': 6,
#   'type': 'track',
#   'uri': 'spotify:track:1ko2lVN0vKGUl9zrU0qSlT'},
#  'currently_playing_type': 'track',
#  'actions': {'disallows': {'pausing': True}},
#  'is_playing': False}

'Just the Two of Us (feat. Bill Withers)'

In [160]:
now_playing_data
df = get_dataframe_from_query("SELECT * from users;")
df  

,id,username,email,password,fitbit_authorized,fitbit_access_token,fitbit_refresh_token,spotify_authorized,spotify_token,spotify_refresh_token
0,1,guest,matpar481@gmail.com,airpod21,True,eyJhbGciOiJIUzI1NiJ9.eyJhdWQiOiIyM1FWWVAiLCJzd...,74d45953842e2bb268c87bb7d27d6533231df9c4140c3b...,True,BQAE53qF9dWlOiZtHEC7X-4NMvGafgK0BqrZYVt8j_gjzu...,AQCZXJOR1tdYhPuqGCTQuAIGLHEZTQ2DSG30M_WKAVRTPL...


In [166]:
import requests
import json
import sys
import os
from src import getSecret, isRunningInCloud, FLASK_PORT, CLOUD_URL

df = get_dataframe_from_query("SELECT * from users;")
# display(df)

# print(df['spotify_token'][0])
# print(df['spotify_refresh_token'][0])

# Use the Spotify Web API to get information about the currently playing track
headers = {
    'Authorization': 'Bearer ' + df['spotify_token'][0]
}
now_playing_url = 'https://api.spotify.com/v1/me/player/currently-playing'
response = requests.get(now_playing_url, headers=headers)
now_playing_data = response.json()

timestamp = now_playing_data['timestamp']
progress_ms = now_playing_data['progress_ms']
album = now_playing_data['item']['album']['name']
duration_ms = now_playing_data['item']['duration_ms']
song = now_playing_data['item']['name']
artist = now_playing_data['item']['artists'][0]['name']
artist_id = now_playing_data['item']['artists'][0]['id']
is_playing = now_playing_data['is_playing']
song_id = now_playing_data['item']['id']

class CurrentSong:
    def __init__(self, song, song_id, artist, artist_id, timestamp, progress_ms, album, duration_ms, is_playing):
        self.song = song
        self.song_id = song_id
        self.artist = artist
        self.artist_id = artist_id
        self.timestamp = timestamp
        self.progress_ms = progress_ms
        self.album = album
        self.duration_ms = duration_ms
        self.is_playing = is_playing
    def __str__(self):
        return f"CurrentSong({self.song}, {self.song_id}, {self.artist}, {self.artist_id}, {self.timestamp}, {self.progress_ms}, {self.album}, {self.duration_ms}, {self.is_playing})"
        
current_song = CurrentSong(song, song_id, artist, artist_id, timestamp, progress_ms, album, duration_ms, is_playing)

df = get_dataframe_from_query("SELECT * from music_history;")
next_song_id = df['song_id'].sample().iloc[0]

# Use the Spotify Web API to start playing a new track
play_url = 'https://api.spotify.com/v1/me/player/play'
play_params = {
    'uris': ['spotify:track:' + next_song_id]
}
response = requests.put(play_url, data=json.dumps(play_params), headers=headers)
# randomly select a song_id



In [151]:
df = get_dataframe_from_query("SELECT * from music_history;")
next_song_id = df['song_id'].sample().iloc[0]

# Use the Spotify Web API to start playing a new track
play_url = 'https://api.spotify.com/v1/me/player/play'
play_params = {
    'uris': ['spotify:track:' + next_song_id]
}
response = requests.put(play_url, data=json.dumps(play_params), headers=headers)

{'error': {'status': 401, 'message': 'The access token expired'}}

In [149]:
df = get_dataframe_from_query("SELECT * from music_history;")
# display(df)
next_song_id = df['song_id'].sample().iloc[0]
print(next_song_id)

27gOj0x9NYYZhaNlSY4nnb


In [119]:
df = get_dataframe_from_query("SELECT * from users;")
display(df)

print(df['spotify_token'][0])
print(df['spotify_refresh_token'][0])
# conn = psycopg2.connect(
#     host=getSecret('SQL_Host'),
#     port=getSecret('SQL_Port'),
#     user=getSecret('SQL_User'),
#     password=getSecret('SQL_Password'),
#     database=getSecret('SQL_Database')
# )
# cur = conn.cursor()
# cur.execute("SELECT * FROM users")
# display(cur.fetchall())
# # cur.execute("UPDATE users SET username='guest' WHERE username='mnann'")
# # retrive user
# cur.execute


# # conn.commit()
# cur.close()
# conn.close()
print(df['spotify_token'][0])
print(df['spotify_refresh_token'][0])

,id,username,email,password,fitbit_authorized,fitbit_access_token,fitbit_refresh_token,spotify_authorized,spotify_token,spotify_refresh_token
0,1,guest,matpar481@gmail.com,airpod21,True,eyJhbGciOiJIUzI1NiJ9.eyJhdWQiOiIyM1FWWVAiLCJzd...,74d45953842e2bb268c87bb7d27d6533231df9c4140c3b...,True,BQA150XEZapjoJWhxDKJ_ZLTHr_X4iv2ndU4Uj8yA20z-H...,AQCZXJOR1tdYhPuqGCTQuAIGLHEZTQ2DSG30M_WKAVRTPL...


BQA150XEZapjoJWhxDKJ_ZLTHr_X4iv2ndU4Uj8yA20z-HX-ckL5kAfpwipfsiqGnDRx_UgCAvL2UlZ6UEOJW2fi9g-aJRKh_RmDAjdMPwOZtk0jYhua3kwj88TTxKJc5OT9U_cuWt7ayOaNhb8KKvhBsqPD7hXTea1sABmSby2i6tCmCDzCt0nrcB8Pm7CVoXDjOdY80goFFskOy18aqD6aU-GwF4QI_ha8Ezzq_58KTCVXdXEUYSZnaKZBaXZIuhCKFNl7FywR1Ak
AQCZXJOR1tdYhPuqGCTQuAIGLHEZTQ2DSG30M_WKAVRTPLBCFAO6rpxZM22hfn8phZEtP8maGSJ8iqq3bqjJNrR8-AJpyHgSxNnmXmYr2yoVL5adVTYTbqRrKbuUIfDFcxE


In [98]:
import sys 
import os
sys.path.append('../..')
from src import getSecret
from src.sql import get_dataframe_from_query, db_connection

df = get_dataframe_from_query("SELECT music_history.*, songs.* FROM music_history JOIN songs ON music_history.song_id = songs.song_id WHERE user_id = {}".format(1))
temp =df['song_id'].iloc[:,0]
df = df.drop(columns=['song_id'])
df.insert(0, 'song_id', temp)
# display(df)
# df['song_id'].unique()

with db_connection() as conn:
    cur = conn.cursor()
    cur.execute("delete from heart_rate where user_id = 1")
    conn.commit()


In [73]:
import psycopg2
conn = psycopg2.connect(
    host=getSecret('SQL_Host'),
    port=getSecret('SQL_Port'),
    user=getSecret('SQL_User'),
    password=getSecret('SQL_Password'),
    database=getSecret('SQL_Database')
)
cur = conn.cursor()
cur.execute("SELECT table_name FROM information_schema.tables WHERE table_name = 'users';")
rows = cur.fetchall()
for row in rows:
    print(row)

cur.execute("SELECT COUNT(*) FROM users")
# cur.execute("Delete from users")
count = cur.fetchone()[0]
print("Number of records in users table:", count)

# # list all tables
# cur.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public'")
# rows = cur.fetchall()
# for row in rows:
#     print(row)

# # fix the query
# cur.execute("SELECT * FROM users")
# rows = cur.fetchall()
# for row in rows:
#     print(row)

# # cur.close()
# # conn.close()

('users',)


ProgrammingError: no results to fetch

In [58]:
import sys
import os
sys.path.append("..")
import pandas as pd

df_heartRate = pd.read_pickle('data/df_heartRate_2023_04_02_to_2023_04_11.pkl')
df_heartRate['datetime'] = pd.to_datetime(df_heartRate['date']) + pd.to_timedelta(df_heartRate['time'])
df_heartRate.rename(columns={'value':'bpm'}, inplace=True)

In [78]:
df_heartRate

import psycopg2

# connect to the database
conn = psycopg2.connect(
    host=getSecret('SQL_Host'),
    port=getSecret('SQL_Port'),
    user=getSecret('SQL_User'),
    password=getSecret('SQL_Password'),
    database=getSecret('SQL_Database')
)

# create a cursor object
cur = conn.cursor()



# execute a query to select all rows from the "Users" table
# cur.execute("SELECT * FROM public.Users")
# cur.execute("SELECT table_schema, table_name FROM information_schema.tables WHERE table_name = 'Users';")
# cur.execute("SET search_path TO public; SELECT * FROM Users")
# cur.execute('SELECT * FROM public.users;')

# for index, row in df_heartRate.iterrows():
#     cur.execute("INSERT INTO public.heart_rate (user_id, bpm, datetime) VALUES (%s, %s, %s)", (1, row['bpm'], row['datetime']))
#     conn.commit()

# cur.execute('SELECT * FROM public.heart_rate;')
# # fetch all rows and print them
# import pandas as pd
# df = pd.DataFrame(cur.fetchall())
# df.columns = [desc[0] for desc in cur.description]
# display(df)
# cur.execute("SELECT music_history.*, songs.* FROM music_history JOIN songs ON music_history.song_id = songs.song_id WHERE user_id = {};".format(1))
cur.execute("SELECT * FROM public.heart_rate;")
# commit
display(pd.DataFrame(cur.fetchall()))
# for row in cur.fetchall():
#     print(row)
# conn.commit()

# df_heartRate = pd.read_sql_query('select * from heartrate where user_id = ' + str(1), con=self.db.engine)
# cur.execute('SELECT * FROM public.music_history;')
# delete 

# cur.execute('SELECT * FROM public.heart_rate;')
# fetch all rows and print them
# rows = cur.fetchall()
# for row in rows:
#     print(row)

# close the cursor and database connection
cur.close()
conn.close()

""


In [27]:
cur.execute("SELECT table_name FROM information_schema.tables WHERE table_name = 'users';")
rows = cur.fetchall()

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


In [21]:
import os
from src import getSecret, isRunningInCloud
# Grabs the folder where the script runs.

DEBUG = True
# Secret key for session management. You can generate random strings here: https://randomkeygen.com/
SECRET_KEY = "o'lEd~n48G[3&@XVF2*]u1`VPF7P%I%,:OA@wuI`.5|%$4neB>h{q=S1N<R5AKL"

if isRunningInCloud():
    SQLALCHEMY_DATABASE_URI = os.environ['DATABASE_URL']
else:
    SQL_username = str(getSecret('SQL_User'))
    SQL_password = str(getSecret('SQL_Password'))
    SQL_host = str(getSecret('SQL_Host'))
    SQL_port = str(getSecret('SQL_Port'))
    SQL_database = str(getSecret('SQL_Database'))
    # Connect to the database
    SQLALCHEMY_DATABASE_URI = 'postgresql://' + SQL_username + ':' + SQL_password + '@' + SQL_host + ':' + SQL_port + '/' + SQL_database
    # 'sqlite:///' + os.path.join(basedir, 'database.db')

    # # Secret key for session management. You can generate random strings here:
    # # https://randomkeygen.com/
    # SECRET_KEY = "o'lEd~n48G[3&@XVF2*]u1`VPF7P%I%,:OA@wuI`.5|%$4neB>h{q=S1N<R5AKL"
    # # Connect to the database
    # SQLALCHEMY_DATABASE_URI = 'sqlite:///' + os.path.join(basedir, 'database.db')


In [175]:
df = get_dataframe_from_query("SELECT * from songs;")
next_song_id = df['song_id'].sample().iloc[0]

# Use the Spotify Web API to start playing a new track
play_url = 'https://api.spotify.com/v1/me/player/play'
play_params = {
    'uris': ['spotify:track:' + next_song_id]
}
response = requests.put(play_url, data=json.dumps(play_params), headers=headers)